In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [3]:
%matplotlib inline

In [4]:
TEXT_COLOUR = {
    'PURPLE':'\033[95m',
    'CYAN':'\033[96m',
    'DARKCYAN':'\033[36m',
    'BLUE':'\033[94m',
    'GREEN':'\033[92m',
    'YELLOW':'\033[93m',
    'RED':'\033[91m',
    'BOLD':'\033[1m',
    'UNDERLINE':'\033[4m',
    'END':'\033[0m'
}

def print_bold(*msgs):
    print(TEXT_COLOUR['BOLD'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_green(*msgs):
    print(TEXT_COLOUR['GREEN'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def print_error(*msgs):
    print(TEXT_COLOUR['RED'])
    print(*msgs)
    print(TEXT_COLOUR['END'])

def wrap_green(msg):
    return TEXT_COLOUR['GREEN'] + msg + TEXT_COLOUR['END']

def wrap_red(msg):
    return TEXT_COLOUR['RED'] + msg + TEXT_COLOUR['END']

def up_down_str(val):
    msg = str(val)
    if val > 0:
        msg = wrap_green(msg)
    elif val < 0:
        msg = wrap_red(msg)
    return msg

In [5]:
exp='roberta-base'
num_layers = 12

In [6]:
tasks = ["CoLA","SST-2","MRPC","STS-B","QQP","MNLI", "MNLI-MM", "QNLI","RTE"]

metrics = {
    "CoLA":["mcc"],
    "MNLI":["acc"],
    "MNLI-MM":["acc"],
    "MRPC":["f1"],
    "QNLI":["acc"],
    "QQP":["f1"],
    "RTE":["acc"],
    "SST-2":["acc"],
    "STS-B":["spearmanr"],
    "WNLI":["acc"] #temp
}

reported_in_paper = {
    "CoLA":0.00,
    "MNLI":0.00,
    "MNLI-MM":0.0,
    "MRPC":0.00,
    "QNLI":0.00,
    "QQP":0.00,
    "RTE":0.00,
    "SST-2":0.00,
    "STS-B":0.00,
    "WNLI":0.00
}

In [7]:

def get_average_val(lines):
    reported = []
    for line in lines:
#         print('\t', line)
        val = float(line.split()[1])
        if val != 0:
            reported.append(val)
    out = 0
    if len(reported) != 0:
        reported.sort(reverse = True)
        candidates = [reported[0]]
        for j in range(1, len(reported)):
            if reported[j] > 0.9 * reported[0]:
                candidates.append(reported[j])
        out = np.mean(candidates)
        
    return out


In [8]:
results = {}

for task in tasks:
    task_results = {}
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        # base metrics
        print(f"../../exp_results/{exp}/{task}/base-{metric}.txt")
        f=open(f"../../exp_results/{exp}/{task}/base-{metric}.txt", "r")
        lines = f.read().splitlines()
        task_results[f'base-{metric}'] = get_average_val(lines)
        
        # no layer metrics
        
        fine_tuning_metrics = []
        f=open(f"../../exp_results/{exp}/{task}/no_layer-{metric}.txt", "r")

        lines = f.read().splitlines()
        fine_tuning_metrics.append(get_average_val(lines))
        
        # fine-tuned metrics
        
        log_file_prefix=''
        for i in reversed(range(int(num_layers/2), num_layers)):
            log_file_prefix += str(i)
            f=open(f"../../exp_results/{exp}/{task}/{log_file_prefix}-{metric}.txt", "r")
            lines = f.read().splitlines()
            fine_tuning_metrics.append(get_average_val(lines))
            
            log_file_prefix +='_'
        
        task_results[f'{metric}'] = list(reversed(fine_tuning_metrics))
        
    results[task] = task_results

../../exp_results/roberta-base/CoLA/base-mcc.txt
../../exp_results/roberta-base/SST-2/base-acc.txt
../../exp_results/roberta-base/MRPC/base-f1.txt
../../exp_results/roberta-base/STS-B/base-spearmanr.txt
../../exp_results/roberta-base/QQP/base-f1.txt
../../exp_results/roberta-base/MNLI/base-acc.txt
../../exp_results/roberta-base/MNLI-MM/base-acc.txt
../../exp_results/roberta-base/QNLI/base-acc.txt
../../exp_results/roberta-base/RTE/base-acc.txt


In [9]:
x_axis = []

for i in range(int(num_layers/2), num_layers):
    x_axis.append(str(i))

x_axis.append("none")

In [10]:
def draw_graph(task, y_label, paper, base, reported):
    plt.figure(figsize=(10,6))
    plt.plot(x_axis, reported)
    
    plt.xlabel("layers")
    plt.ylabel(y_label)
    
    if paper == 0.0:    
        gap = max(reported) - min(reported)
        top = max(max(reported), base) + (gap*0.2)
        bottom = min(min(reported), base) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
    else:
        gap = max(reported) - min(reported)
        top = max(max(reported), base, paper) + (gap*0.2)
        bottom = min(min(reported), base, paper) - (gap*0.2)
    
        plt.ylim(bottom, top)

        plt.axhline(y=base, linestyle='--', c='green')
        plt.axhline(y=paper, linestyle='--', c='red')
    
    plt.title(f'{exp}-{task} ({round(base,4)})')
    plt.savefig(f'images/{exp}/{task}', format='png', bbox_inches='tight')
    plt.show()

In [11]:
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        reported = task_results[metric]
        base = task_results[f'base-{metric}']
        print_bold(task, metric, ': b -', round(base * 100, 2), 'h -',round(task_results[metric][0] * 100, 2), 'n -', round(task_results[metric][-1] * 100, 2))


CoLA mcc : b - 59.53 h - 58.54 n - 0


SST-2 acc : b - 94.31 h - 94.0 n - 80.17


MRPC f1 : b - 92.28 h - 90.78 n - 81.22


STS-B spearmanr : b - 90.64 h - 88.85 n - 19.68


QQP f1 : b - 88.84 h - 87.7 n - 62.51


MNLI acc : b - 87.36 h - 85.71 n - 52.61


MNLI-MM acc : b - 87.05 h - 85.88 n - 53.03


QNLI acc : b - 92.75 h - 91.71 n - 65.71


RTE acc : b - 77.51 h - 75.45 n - 57.54



In [12]:
import copy 

layer_90 = []
layer_95 = []

threshold_90 = 0.9
threshold_95 = 0.95
x_axis.reverse()

for task in tasks:
#     print_bold(task)
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        flag_90 = True
        flag_95 = True

        if task != "CoLA":
            for ind, val in enumerate(reported):

                if val/base > threshold_90 and flag_90:
                    flag_90 = False
                    layer_90.append(ind)

                if val/base > threshold_95 and flag_95:
                    flag_95 = False
                    layer_95.append(ind)

            if flag_90:
                print(task, "Fails to achieve 90% threshold", reported[-1]/base)
                layer_90.append(len(reported)-1)

            if flag_95:
                print(task, "Fails to achieve 95% threshold", reported[-1]/base)
                layer_95.append(len(reported)-1)


            
print(x_axis)
            
            
print(layer_90)
min_layer_ind_90 = max(layer_90)
print("layer_90 ", min_layer_ind_90, 'layer:', x_axis[min_layer_ind_90], round((1-(min_layer_ind_90/num_layers)) * 100, 2), '%')

print(layer_95)
min_layer_ind_95 = max(layer_95)
print("layer_95 ", min_layer_ind_95, 'layer:', x_axis[min_layer_ind_95], round((1-(min_layer_ind_95/num_layers)) * 100, 2), '%')


firsts = []
seconds = []
    
for task in tasks:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        base = task_results[f'base-{metric}']
        reported = copy.deepcopy(task_results[metric])
        reported.reverse()
        
        if task != "CoLA":
            first = round(100*reported[0]/base, 2)
            second = round(100*reported[1]/base, 2)
            firsts.append(first)
            seconds.append(second)
            
        print_bold(task, base)
        print('\t90', reported[min_layer_ind_90], round(reported[min_layer_ind_90]/base * 100, 2))
        print('\t95', reported[min_layer_ind_95], round(reported[min_layer_ind_95]/base * 100, 2))
        
print_bold(len(firsts), np.mean(firsts), np.mean(seconds), round(np.mean(seconds) - np.mean(firsts),2))

['none', '11', '10', '9', '8', '7', '6']
[1, 1, 1, 1, 2, 2, 1, 5]
layer_90  5 layer: 7 58.33 %
[1, 2, 3, 2, 3, 2, 2, 6]
layer_95  6 layer: 6 50.0 %

CoLA 0.5953106049687354

	90 0.5857252464799216 98.39
	95 0.5854127107560999 98.34

SST-2 0.9431192660550458

	90 0.9327981651376147 98.91
	95 0.9400229357798164 99.67

MRPC 0.9227957716915123

	90 0.8960831667117715 97.11
	95 0.9077645199872366 98.37

STS-B 0.9063766246945144

	90 0.876466358072318 96.7
	95 0.888478113238167 98.03

QQP 0.888386018735033

	90 0.874658248225866 98.45
	95 0.8769683245580799 98.71

MNLI 0.8736220071319408

	90 0.8492715231788079 97.21
	95 0.8571370351502802 98.11

MNLI-MM 0.8705451586655817

	90 0.8499389747762407 97.63
	95 0.8587672904800652 98.65

QNLI 0.9275489657697236

	90 0.9135273659161631 98.49
	95 0.91706022332052 98.87

RTE 0.7751418256833419

	90 0.7245487364620938 93.47
	95 0.7545126353790612 97.34

8 66.41499999999999 90.51249999999999 24.1



In [13]:
for task in ["STS-B"]:
    task_results = results[task]
    task_metrics = metrics[task]
    for metric in task_metrics:
        
        print(task_results[metric][-1])
        print(task_results[metric][-2])

0.19678919023438018
0.8394335349615727


In [14]:
latex_metrics = {
    "CoLA":"MCC",
    "MNLI":"Acc.",
    "MNLI-MM":"Acc.",
    "MRPC":"F$_1$",
    "QNLI":"Acc.",
    "QQP":"F$_1$",
    "RTE":"Acc.",
    "SST-2":"Acc.",
    "STS-B":"$\\rho$"
}

In [15]:
print("\\begin{center}\n\t\\scalebox{0.88}{\n\t\t\\begin{tabular}{rc|ccccccc} \n\t\t\\toprule[1pt] \n\t\t\\multirow{2}{*}{Task (metric)} & \\multirow{2}{*}{Baseline} & \\multicolumn{7}{c}{Fine-tuned layers} \\\\ \n\t\t\\cline{3-9} \n\t\t& & 6-11 & 7-11 & 8-11 & 9-11 & 10-11 & 11-11 & None \\\\ \n\t\t\t\\midrule")

avg_performance = []

for task in tasks:
    m = metrics[task][0]
    base_key = f"base-{m}"
    
    if task == "MNLI-MM":
        row = f"\t\t\tMNLI-mm ({latex_metrics[task]}) & "
    else:
        row = f"\t\t\t{task} ({latex_metrics[task]}) & "
    
    row += "{:0.2f}".format(round(results[task][base_key] * 100, 2))
    
    for ind, val in enumerate(results[task][m]):
        row += " & {:0.2f}".format(round(val * 100,2))
        
        if len(avg_performance) == ind:
            avg_performance.append([])
            
            
        percent = (val / results[task][base_key]) * 100
        avg_performance[ind].append(percent)
        
#     row += "& {}".format(results[task]["90%"])
#     row += "& {}".format(results[task]["95%"])
        
    row += " \\\\"
    print(row)
    
print("\t\t\t\\midrule\\midrule")

row = "\t\t\tRel. perf. (\%) & 100.00"

for perf in avg_performance:
    row += " & {:0.2f}".format(round(np.mean(perf) ,2))
    
row += " \\\\"

print(row)
    
print("\t\t\\end{tabular}\n\t}\n\t\\caption{Vanilla-RoBERTa-base on GLUE}\n\t\\label{table:finetune-all}\n\\end{center}")

\begin{center}
	\scalebox{0.88}{
		\begin{tabular}{rc|ccccccc} 
		\toprule[1pt] 
		\multirow{2}{*}{Task (metric)} & \multirow{2}{*}{Baseline} & \multicolumn{7}{c}{Fine-tuned layers} \\ 
		\cline{3-9} 
		& & 6-11 & 7-11 & 8-11 & 9-11 & 10-11 & 11-11 & None \\ 
			\midrule
			CoLA (MCC) & 59.53 & 58.54 & 58.57 & 55.19 & 54.26 & 52.62 & 51.53 & 0.00 \\
			SST-2 (Acc.) & 94.31 & 94.00 & 93.28 & 93.49 & 93.58 & 93.18 & 92.02 & 80.17 \\
			MRPC (F$_1$) & 92.28 & 90.78 & 89.61 & 88.86 & 88.74 & 87.98 & 85.27 & 81.22 \\
			STS-B ($\rho$) & 90.64 & 88.85 & 87.65 & 87.17 & 86.84 & 85.49 & 83.94 & 19.68 \\
			QQP (F$_1$) & 88.84 & 87.70 & 87.47 & 87.44 & 87.08 & 85.62 & 83.11 & 62.51 \\
			MNLI (Acc.) & 87.36 & 85.71 & 84.93 & 85.04 & 84.59 & 82.51 & 76.22 & 52.61 \\
			MNLI-mm (Acc.) & 87.05 & 85.88 & 84.99 & 85.62 & 85.07 & 83.27 & 78.00 & 53.03 \\
			QNLI (Acc.) & 92.75 & 91.71 & 91.35 & 90.99 & 90.70 & 89.24 & 84.93 & 65.71 \\
			RTE (Acc.) & 77.51 & 75.45 & 72.45 & 68.63 & 66.97 & 64.95 & 61